**Общая информация**

**Срок сдачи:** 29 марта 2021, 08:30 

**Штраф за опоздание:** по 1 баллу за 24 часа задержки. Через 5 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0220, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
2. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [97]:
import numpy as np
import matplotlib.pyplot as plt


import seaborn as sns
from sklearn import datasets
from sklearn.base import BaseEstimator
#!pip install sklearn.datasets.fetch_mldata
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import euclidean_distances
from sklearn.neighbors import KDTree
from sklearn.datasets import fetch_openml, fetch_20newsgroups

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity

import operator

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. 
Для подсчета расстояний можно использовать функции [отсюда](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

In [157]:
class MyKNeighborsClassifier(BaseEstimator):
    
    def __init__(self, n_neighbors = 2, algorithm="brute", metric = "euclidean"):
        self.metric = metric
        self.n_neighbors = n_neighbors
        self.algorithm_bool = False
        self.algorithm = algorithm
        self.my_X = []
        self.my_Y = []

        if algorithm == "kd_tree":
            self.algorithm_bool = True
        return None
        '''
        elif algorithm == "brute":
            self.algorithm_bool = False
        elif algorithm == None:
            self.algorithm_bool = False
        else:
            raise ValueError("Algorithm is not supported", algorithm)
        '''

    def fit(self, X, y):
        X, y = check_X_y(X, y)
        self.my_X = X
        self.my_Y = y

    
    def predict(self, X):
        #check_is_fitted(self)
        X = check_array(X)

        if self.algorithm_bool:
            kdt = KDTree(self.my_X, metric="euclidean")
            B, A = kdt.query(X, self.n_neighbors)

        elif self.metric == "euclidean":
            A = np.argsort(euclidean_distances(X, self.my_X), axis=1)[:,:self.n_neighbors]
            B = np.sort(euclidean_distances(X, self.my_X), axis=1)[:,:self.n_neighbors]
        else:
            #print("i'm here")
            A = np.argsort(cosine_similarity(X, self.my_X), axis=1)[:,:self.n_neighbors]
            B = np.sort(cosine_similarity(X, self.my_X), axis=1)[:,:self.n_neighbors]

        C = []
        for j in range(A.shape[0]):
            D = {}
            for i in range(self.n_neighbors):
                if A[j, i] in D:
                    D[A[j, i]] += B[j, i]
                else:
                    D[A[j, i]] = B[j, i]
            C.append(max(D.items(), key=operator.itemgetter(1))[0])

        return self.my_Y[C]
    

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [158]:
iris = datasets.load_iris()

In [159]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.25, stratify=iris.target)

In [160]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier()

In [161]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [162]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
accuracy_score(y_test, my_clf_pred), accuracy_score(y_test, sklearn_pred )
#assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

(0.9473684210526315, 0.9736842105263158)

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. 

In [163]:
%time clf.fit(X_train, y_train)

CPU times: user 1.21 ms, sys: 26 µs, total: 1.24 ms
Wall time: 1.43 ms


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [164]:
%time my_clf.fit(X_train, y_train)

CPU times: user 352 µs, sys: 25 µs, total: 377 µs
Wall time: 333 µs


In [165]:
%time clf.predict(X_test)

CPU times: user 4.22 ms, sys: 0 ns, total: 4.22 ms
Wall time: 4.37 ms


array([0, 2, 0, 0, 2, 2, 2, 1, 0, 2, 0, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1,
       0, 2, 1, 0, 1, 2, 2, 1, 2, 2, 1, 0, 0, 0, 0, 2])

In [166]:
%time my_clf.predict(X_test)

CPU times: user 1.92 ms, sys: 762 µs, total: 2.68 ms
Wall time: 6.83 ms


array([0, 2, 0, 0, 2, 2, 2, 1, 0, 2, 0, 1, 1, 1, 0, 1, 1, 2, 2, 1, 1, 1,
       0, 2, 1, 0, 1, 2, 2, 1, 2, 2, 1, 0, 0, 0, 0, 2])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict сравнимой со sklearn для iris. Допускается замедление не более чем в 2 раза. 
Для этого используем numpy. Точность не должна уступать значению KNN из sklearn. 

In [167]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [168]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [169]:
%time clf.fit(X_train, y_train)

CPU times: user 1.08 ms, sys: 0 ns, total: 1.08 ms
Wall time: 1.25 ms


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [170]:
%time my_clf.fit(X_train, y_train)

CPU times: user 347 µs, sys: 24 µs, total: 371 µs
Wall time: 380 µs


In [171]:
%time clf.predict(X_test)

CPU times: user 2.23 ms, sys: 16 µs, total: 2.24 ms
Wall time: 5.61 ms


array([2, 2, 0, 2, 0, 1, 0, 1, 0, 1, 2, 0, 2, 1, 2])

In [172]:
%time my_clf.predict(X_test)

CPU times: user 816 µs, sys: 57 µs, total: 873 µs
Wall time: 1.18 ms


array([2, 2, 0, 2, 0, 1, 0, 1, 0, 1, 2, 0, 2, 1, 2])

In [173]:
sklearn_pred = clf.predict(X_test)
my_clf_pred = my_clf.predict(X_test)
#accuracy_score(y_test, my_clf_pred), accuracy_score(y_test, sklearn_pred )
assert abs( accuracy_score(y_test, my_clf_pred) -  accuracy_score(y_test, sklearn_pred ) )<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [174]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [175]:
data = newsgroups['data']
target = newsgroups['target']
#data, target


Преобразуйте текстовые данные из data с помощью [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Словарь можно ограничить по частотности.

In [178]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english', min_df=2, max_df=10, max_features = 10000)
my_X = vectorizer.fit_transform(data)
my_y = target
#print(X)
print(my_X.shape, my_y.shape)


(11314, 10000) (11314,)


*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

In [179]:
from sklearn.model_selection import cross_validate
clf = MyKNeighborsClassifier()
x3 = cross_validate(estimator=clf, X=my_X, y=my_y, cv=3, scoring="accuracy")
x3

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.

  FitFailedWarning)


{'fit_time': array([0.001441  , 0.00121069, 0.0008626 ]),
 'score_time': array([0., 0., 0.]),
 'test_score': array([nan, nan, nan])}

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальную точность в среднем на валидации на 3 фолдах).
Постройте график зависимости средней точности от количества соседей. Можно рассмотреть число соседей от 1 до 10.

In [96]:
import matplotlib.pyplot as plt

n_neighbors_max = 10
A, B = [], []

X=X.toarray()

for i in range(1, n_neighbors_max+1):
    print(i)
    clf = MyKNeighborsClassifier(n_neighbors=i, algorithm="brute")
    my_dict = cross_validate(estimator=clf, X=X, y=y, cv=3, scoring="accuracy")
    A.append(i)
    B.append(my_dict['test_score'].mean())

plt.plot(A, B)
plt.xlabel('Number of neighbors')
plt.ylabel('Score')
plt.title('Dependence of the average accuracy on the number of neighbors ')
plt.show()

1


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


TypeError: ignored

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf векторизацию( sklearn.feature_extraction.text.TfidfVectorizer)

Сравните модели, выберите лучшую.

In [ ]:

for i in range(1, n_neighbors_max+1):
    print(i)
    clf = MyKNeighborsClassifier(n_neighbors=i, metric=cosine_similarity)
    my_dict = cross_validate(estimator=clf, X=X, y=y, cv=3, scoring="accuracy")
    A.append(i)
    B.append(my_dict['test_score'].mean())

plt.plot(A, B)
plt.xlabel('Number of neighbors')
plt.ylabel('Score')
plt.title('Dependence of the average accuracy on the number of neighbors ')
plt.show()

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим точность вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.